# A Brief Introduction

This notebook will guide you though several examples that demonstrate how you can use `idom` to create interactive interfaces for websites, analytic dashboard, and so much more. We'll learn the basics of user interaction by creating a simple slideshow, but move on to creating games and plots with [Matplotlib](https://matplotlib.org):

1. [Slideshows and Basic Interactivity](#Slideshows-and-User-Interaction)

2. [Plotting with Matplotlib](#Plotting-with-Matplotlib)

3. [The Game Snake](#The-Game-Snake)

In [ ]:
import idom

### A Little Set Up

Since you may be running this notebook on [mybinder.org](https://mybinder.org) we have to do a little bit of work to make sure that your frontend view connects to the right URL. You shouldn't have to change anything here. Be sure to [post an issue](https://github.com/rmorshea/idom/issues) if you experience any problems. 

In [ ]:
from example_utils import localhost, pretty_dict_string

webpage_url = localhost('http', 8765) + '/idom/client/index.html'
websocket_url = localhost('ws', 8765) + '/idom/stream'

show, element = idom.hotswap()
idom.SimpleServer(element).daemon("0.0.0.0", 8765, access_log=False)

# Slideshows and User Interaction

Let's use iDOM to create a simple slideshow which changes whenever a user clicks an image. To do this we define a function called `Slideshow` which will return an `<img/>` element. Attached to this `<img/>` is a handler for `onClick` events created whenever a user clicks the image.

In [ ]:
@idom.element
async def Slideshow(self, index=0):
    events = idom.Events()

    @events.on("click")
    async def change():
        self.update(index + 1)

    url = f"https://picsum.photos/800/300?image={index}"
    return idom.node("img", src=url, eventHandlers=events)

## Try it Out!

The following cells will display the output here. However you can also see the same thing in a standard web page:

In [ ]:
print('Get a webpage that streams layout updates via a websocket:')
print(webpage_url)

If you just want to see it here run the cell below...

In [ ]:
print("Try clicking the image! 🖱️")

show(Slideshow)

idom.display("jupyter", websocket_url)

## What Just Happened?

At the core of iDOM are simple dictionary structures that model an HTML layout.

To help you understand what's happening let's see what this dictionary model looks like:

In [ ]:
slideshow = Slideshow()
model = await slideshow.render()
print(pretty_dict_string(model))

This will render the following HTML

```html
<img src='https://picsum.photos/800/300?image={index}' onClick={slideshow-change-callback}/>
```

Where each time the user clicks the rendered image:

1. The `slideshow-change-callback` is triggered and calls `change()`

2. The `change()` function then tells the `Slideshow` element to update with a new `{index}`.

3. A new image is rendered for the user with the updated `src` index.

# Plotting with Matplotlib

The following example shows how you might create a matplotlib plot which can be updated based on user interactions. In this simple case the plot updates each time a user clicks on the graph, however you can imaging more complex scenarios in which users select datasets from drop down menus or interact with sliders.

In [ ]:
from matplotlib import pyplot as plt

At the moment `idom` doesn't have native support for matplotlib plots, however you can still render an image by:

1. Creating an `Image` element with some format (e.g. SVG).

2. Calling a figure's `savefig` method and writing to the `Image.io` attribute.

In [ ]:
import random


@idom.element
async def Figure(self):
    x, y = random_walk(200)
    img = plot(x, y)

    events = idom.Events()
    
    @events.on("click")
    async def scramble():
        self.update()

    return idom.html.div(img, eventHandlers=events)


def plot(*args, **kwargs):
    fig, axes = plt.subplots()
    axes.plot(*args, **kwargs)
    img = idom.Image("svg")
    fig.savefig(img.io, format="svg")
    plt.close(fig)
    return img


def random_walk(n):
    y = []
    current = 0
    x = range(n)
    for _ in range(len(x)):
        current += (random.random() - 0.5)
        y.append(current)
    return x, y

## Try it Out

In [ ]:
print("Try clicking the plot! 📈")

show(Figure)

idom.display("jupyter", websocket_url)

# The Game Snake

Ok, let's take a look at something a little more complicated.

In [ ]:
import enum
import time
import random
import asyncio


class WASD(enum.Enum):
    w = (-1, 0)
    a = (0, -1)
    s = (1, 0)
    d = (0, 1)


@idom.element(state="grid_size, block_size")
async def Game(self, grid_size, block_size):
    game = idom.StaticBunch(
        snake=[],
        grid=Grid(grid_size, block_size),
        new_direction=idom.Var(WASD.d),
        old_direction=idom.Var(WASD.d),
        food=idom.Var(None),
        won=idom.Var(False),
        lost=idom.Var(False),
    )

    @game.grid.eventHandlers.on("KeyDown")
    async def direction_change(key):
        if hasattr(WASD, key):
            game.new_direction.set(WASD[key])

    game.snake.extend([
        (grid_size // 2 - 1, grid_size // 2 - 3),
        (grid_size // 2 - 1, grid_size // 2 - 2),
        (grid_size // 2 - 1, grid_size // 2 - 1),
    ])

    grid_points = set((x, y) for x in range(grid_size) for y in range(grid_size))

    def set_new_food():
        points_not_in_snake = grid_points.difference(game.snake)
        new_food = random.choice(list(points_not_in_snake))
        get_grid_block(game.grid, new_food).update("blue")
        game.food.set(new_food)

    @self.animate
    async def loop():
        if game.won.get() or game.lost.get():
            self.update()
            await asyncio.sleep(1)
        else:
            await draw(game, grid_size, set_new_food)
            self.animate(loop)

    set_new_food()
    return game.grid


async def draw(game, grid_size, set_new_food):
    start = time.time()
    
    if game.snake[-1] in game.snake[:-1]:
        # point out where you touched
        get_grid_block(game.grid, game.snake[-1]).update("red")
        game.lost.set(True)
        return

    vector_sum = tuple(map(sum, zip(
        game.old_direction.get().value,
        game.new_direction.get().value
    )))
    if vector_sum != (0, 0):
        game.old_direction.set(game.new_direction.get())

    new_head = (
        # grid wraps due to mod op here
        (game.snake[-1][0] + game.old_direction.get().value[0]) % grid_size,
        (game.snake[-1][1] + game.old_direction.get().value[1]) % grid_size,
    )

    game.snake.append(new_head)

    if new_head == game.food.get():
        if len(game.snake) == grid_size * grid_size:
            game.won.set(True)
            return
        set_new_food()
    else:
        get_grid_block(game.grid, game.snake.pop(0)).update("white")
    
    # update head after tail - new head may be the same as the old tail
    get_grid_block(game.grid, new_head).update("black")

    stop = time.time()
    await asyncio.sleep(0.5 - (start - stop))


def Grid(grid_size, block_size):
    return idom.html.div(
        [
            idom.html.div(
                [Block("white", block_size) for i in range(grid_size)],
                style={"height": block_size},
            )
            for i in range(grid_size)
        ],
        style={
            "height": f"{block_size * grid_size}px",
            "width": f"{block_size * grid_size}px",
        },
        eventHandlers=idom.Events(),
        tabIndex=-1,
    )


@idom.element(state="block_size")
async def Block(self, color, block_size):
    return idom.html.div(
        style={
            "height": f"{block_size}px",
            "width": f"{block_size}px",
            "backgroundColor": color,
            "display": "inline-block",
            "border": "1px solid white",
        }
    )


def get_grid_block(grid, point):
    x, y = point
    return grid["children"][x]["children"][y]

In [ ]:
print("Click to start playing 🎮")
print()
print("Slow internet may cause inconsistent frame pacing 😢")

show(Game, 15, 30)

idom.display("jupyter", websocket_url)